In [14]:
import pandas as pd
import numpy as np
import jieba
import torch
import tqdm
import nltk
import pickle
import os
import regex as re
import collections
from opencc import OpenCC

In [3]:
from pytorch_pretrained_bert import BertTokenizer

### First load your data

In [4]:
# filenames = os.listdir("data")

# df_list = []

# for filename in filenames:
#     filename = "data/" + filename
#     df = pd.read_csv(filename)
#     df_list.append(df)
# df = pd.concat(df_list, axis=0, ignore_index=True)

### Some Data Cleaning Functions

The minimal is that you need **filterPunctuation** for you dataset.

In [18]:
def getPercent(x):
    return [float(item.split("=")[1][:-1]) for item in x]

# This one is the most imporant
def filterPunctuation(x):
    x = re.sub(r'[‘’]', "'", x)
    x = re.sub(r'[“”]', '"', x)
    x = re.sub(r'[…]', '...', x)
    x = re.sub(r'[—]', '-', x)
    return x

def filtering(x):
    pattern = r"[^\u4e00-\u9fff0-9A-Za-z\s+\.\!\/_,：:;-|$%^*()+\"\'+——！，。？、《》“”~@#￥%…&*（）]+"
    return re.sub(pattern, "", x)

def removeURL(x):
    return re.sub(r'https?:\/\/[A-Za-z0-9.\/\-]*', '', x)

cc = OpenCC('t2s')

def clean_data(x):
    x = filterPunctuation(x.strip()).replace("\xa0", "").replace(" ", "").lower()
    x = cc.convert(x)
    return x

In [19]:
df_clean = pd.read_json("clean_data/clean_data.json")

In [20]:
contents = [clean_data(item[0]) for item in df_clean["summarization"]]
origin_titles = [clean_data(item) for item in df_clean["origin_title"]]
modified_titles = [clean_data(item) for item in df_clean["third_title"]]

In [21]:
all_indices = []

for idx in range(len(contents)):
    if len(contents[idx]) < 100:
        continue
    
    if origin_titles[idx] == modified_titles[idx]:
        continue
        
    all_indices.append(idx)

contents = [contents[idx] for idx in all_indices]
origin_titles = [origin_titles[idx] for idx in all_indices]
modified_titles = [modified_titles[idx] for idx in all_indices]

In [22]:
df_save = pd.DataFrame({"contents":contents, 
                          "origin_titles":origin_titles, 
                          "modified_titles":modified_titles})

split your data

In [23]:
mask = np.random.rand(len(df_save)) < 0.8

Save your data

In [25]:
df_save[mask].to_json("clean_data/train.json", force_ascii=False)
df_save[~mask].to_json("clean_data/dev.json", force_ascii=False)

## If you finish the above code, start from here!

load data

In [27]:
df_train = pd.read_json("clean_data/train.json")
df_val = pd.read_json("clean_data/dev.json")

define a tokenizer

In [52]:
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")

Turn your data into tensors: input_ids, mask, and token types

In [84]:
train_data = []

encoder_max = 220
decoder_max = 40

for idx in tqdm.tqdm_notebook(range(len(df_train))):
    
    content = df_train.iloc[idx]['contents']
    origin = df_train.iloc[idx]['origin_titles']
    third = df_train.iloc[idx]['modified_titles']
    
    content =  tokenizer.tokenize(content)
    origin = ["[CLS]"] + tokenizer.tokenize(origin) + ["[SEP]"]
    
    encoder_input = origin + content
    encoder_input = encoder_input[:encoder_max-1] + ["[SEP]"]
    encoder_type_ids = [0] * len(origin) + [1] * (len(encoder_input) - len(origin))

    # modified title
    third = ["[CLS]"] + tokenizer.tokenize(third)
    third = third[:decoder_max-1] + ["[SEP]"]

    # mask
    mask_encoder_input = [1] * len(encoder_input)
    mask_third = [1] * len(third)
    
    # conver to ids
    encoder_input = tokenizer.convert_tokens_to_ids(encoder_input)
    third = tokenizer.convert_tokens_to_ids(third)
    
    # padding
    encoder_input += [0] * (encoder_max - len(encoder_input))
    third += [0] * (decoder_max - len(third))
    
    mask_encoder_input += [0] * (encoder_max - len(mask_encoder_input))
    mask_third += [0] * (decoder_max - len(mask_third))
    
    encoder_type_ids += [0] * (encoder_max - len(encoder_type_ids))
    
    # modified type ids
    third_type_ids = torch.zeros(len(mask_third)).long()

    # turn into tensor
    encoder_input = torch.LongTensor(encoder_input)
    third = torch.LongTensor(third)
    
    mask_encoder_input = torch.LongTensor(mask_encoder_input)
    mask_third = torch.LongTensor(mask_third)
    
    encoder_type_ids = torch.LongTensor(encoder_type_ids)
    third_type_ids = third_type_ids
    
    train_data.append((encoder_input, 
                       third, 
                       mask_encoder_input, 
                       mask_third,
                       encoder_type_ids,
                       third_type_ids))

In [85]:
encoder_input, \
        third, \
        mask_encoder_input, \
        mask_third, \
        encoder_type_ids, \
        third_type_ids = zip(*train_data)

In [88]:
encoder_input = torch.stack(encoder_input)
third = torch.stack(third)
mask_encoder_input = torch.stack(mask_encoder_input)
mask_third = torch.stack(mask_third)
encoder_type_ids = torch.stack(encoder_type_ids)
third_type_ids = torch.stack(third_type_ids)

In [89]:
train_data = [encoder_input, \
        third, \
        mask_encoder_input, \
        mask_third, \
        encoder_type_ids, \
        third_type_ids]

Save the processed data. Note that it can be loaded by pickle as well.

In [90]:
torch.save(train_data, "train_data.pth")

## Validation

In [91]:
val_data = []

encoder_max = 220
decoder_max = 40

for idx in tqdm.tqdm_notebook(range(len(df_val))):
    
    content = df_val.iloc[idx]['contents']
    origin = df_val.iloc[idx]['origin_titles']
    third = df_val.iloc[idx]['modified_titles']
    
    content =  tokenizer.tokenize(content)
    origin = ["[CLS]"] + tokenizer.tokenize(origin) + ["[SEP]"]
    
    encoder_input = origin + content
    encoder_input = encoder_input[:encoder_max-1] + ["[SEP]"]
    encoder_type_ids = [0] * len(origin) + [1] * (len(encoder_input) - len(origin))

    # modified title
    third = ["[CLS]"] + tokenizer.tokenize(third)
    third = third[:decoder_max-1] + ["[SEP]"]

    # mask
    mask_encoder_input = [1] * len(encoder_input)
    mask_third = [1] * len(third)
    
    # conver to ids
    encoder_input = tokenizer.convert_tokens_to_ids(encoder_input)
    third = tokenizer.convert_tokens_to_ids(third)
    
    # padding
    encoder_input += [0] * (encoder_max - len(encoder_input))
    third += [0] * (decoder_max - len(third))
    
    mask_encoder_input += [0] * (encoder_max - len(mask_encoder_input))
    mask_third += [0] * (decoder_max - len(mask_third))
    
    encoder_type_ids += [0] * (encoder_max - len(encoder_type_ids))
    
    # modified type ids
    third_type_ids = torch.zeros(len(mask_third)).long()

    # turn into tensor
    encoder_input = torch.LongTensor(encoder_input)
    third = torch.LongTensor(third)
    
    mask_encoder_input = torch.LongTensor(mask_encoder_input)
    mask_third = torch.LongTensor(mask_third)
    
    encoder_type_ids = torch.LongTensor(encoder_type_ids)
    third_type_ids = third_type_ids
    
    val_data.append((encoder_input, 
                       third, 
                       mask_encoder_input, 
                       mask_third,
                       encoder_type_ids,
                       third_type_ids))

In [92]:
encoder_input, \
        third, \
        mask_encoder_input, \
        mask_third, \
        encoder_type_ids, \
        third_type_ids = zip(*val_data)

In [93]:
encoder_input = torch.stack(encoder_input)
third = torch.stack(third)
mask_encoder_input = torch.stack(mask_encoder_input)
mask_third = torch.stack(mask_third)
encoder_type_ids = torch.stack(encoder_type_ids)
third_type_ids = torch.stack(third_type_ids)

In [94]:
val_data = [encoder_input, \
        third, \
        mask_encoder_input, \
        mask_third, \
        encoder_type_ids, \
        third_type_ids]

In [95]:
torch.save(val_data, "val_data.pth")